In [41]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import re

In [2]:
pricedf = pd.read_csv('src/scraping/atx_listings_trulia.csv')

In [6]:
geodf = pd.read_csv('data/Austin_addresses.csv')

/Users/austinmadert/anaconda3/envs/lihenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
geodf.head()

,address,address1,address2,c_street_number,c_street,c_unit,c_city,c_state,c_zip_code,g_street_number,g_street,g_unit,g_city,g_state,g_zip_code,g_county,lat,lon
0,20813 PENNY ROYAL DR PFLUGERVILLE TX 78660,20813 PENNY ROYAL DR PFLUGERVILLE TX 78660,PENNY ROYAL DR PFLUGERVILLE TX 78660,20813,PENNY ROYAL DRIVE,NaN,PFLUGERVILLE,TX,78660.0,20813,Penny Royal Drive,NaN,Pflugerville,Texas,78660.0,Travis County,30.484112,-97.563114
1,10604 FOUNDATION ROAD AUS1 TN TX 78726,10604 FOUNDATION RD AUS1 TN TX 78726,FOUNDATION RD AUS1 TN TX 78726,10604,FOUNDATION ROAD,NaN,AUSTIN,TX,78726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12200 PRATOLINA DR AUSTIN TX 78739,12200 PRATOLINA DR AUSTIN TX 78739,PRATOLINA DR AUSTIN TX 78739,12200,PRATOLINA DRIVE,NaN,AUSTIN,TX,78739.0,12200,Pratolina Drive,NaN,Austin,Texas,78739.0,Travis County,30.186914,-97.928031
3,311 EVENING GROSBEAK DRIVE PFLUGERVILLE TX 78660,311 EVENING GROSBEAK DR PFLUGERVILLE TX 78660,EVENING GROSBEAK DR PFLUGERVILLE TX 78660,311,EVENING GROSBEAK DRIVE,NaN,PFLUGERVILLE,TX,78660.0,311,Evening Grosbeak Drive,NaN,Pflugerville,Texas,78660.0,Travis County,30.446861,-97.627389
4,8201 EDGEMOOR PLACE AUSTIN TX 78749,NaN,NaN,8201,EDGEMOOR PLACE,NaN,AUSTIN,TX,78749.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pricedf.head()

,address,bathrooms,bedrooms,city_state_zip,house_type,price,sqft,url
0,4159 Steck Ave #102,2 Bathrooms,3 Bedrooms,"Austin, TX 78759",Condo,"$280,000","1,427 Square Feet",https://www.trulia.com/p/tx/austin/4159-steck-...
1,15810 De Peer Cv,3 Bathrooms,4 Bedrooms,"Austin, TX 78717",Single-Family Home,"$299,000","2,224 Square Feet",https://www.trulia.com/p/tx/austin/15810-de-pe...
2,8201 Lazy Ln,1 Bathroom,3 Bedrooms,"Austin, TX 78757",Single-Family Home,"$329,900","1,005 Square Feet",https://www.trulia.com/p/tx/austin/8201-lazy-l...
3,14511 Fitzgibbon Dr,2 Bathrooms,3 Bedrooms,"Austin, TX 78725",Single-Family Home,"$229,874","1,499 Square Feet",https://www.trulia.com/p/tx/austin/14511-fitzg...
4,10304 Bankhead Dr,2 Bathrooms,3 Bedrooms,"Austin, TX 78757",Single-Family Home,"$237,501","1,295 Square Feet",https://www.trulia.com/p/tx/austin/10304-bankh...


In [29]:
pricedf['price'] = pricedf['price'].map(lambda x: str(x).replace(',', '').replace('$',''))

In [31]:
pricedf['sqft'] = pricedf['sqft'].map(lambda x: str(x).replace(',', '').replace(' Square Feet',''))

In [33]:
pricedf['sqft'].fillna(0, inplace=True)

In [36]:
pricedf['sqft'] = pricedf['sqft'].map(lambda x: 0 if x == 'nan' else x)

In [40]:
pricedf.house_type.unique()

array(['Condo', 'Single-Family Home', 'Mobile/Manufactured', 'Townhouse',
       nan, 'Lot/Land', '4,267 Square Feet', 'Lot Size: 9060 sqft',
       'Lot Size: 9496 sqft', '456 Square Feet', 'Lot Size: 8494 sqft',
       'Lot Size: 10541 sqft', 'Lot Size: 1.04 acres',
       'Lot Size: 0.61 acres', 'Lot Size: 1.00 acres', '952 Square Feet',
       'Lot Size: 1.53 acres', 'Lot Size: 0.29 acres',
       '3,565 Square Feet', 'Lot Size: 8973 sqft', '4,089 Square Feet',
       'Lot Size: 9583 sqft', 'Lot Size: 8145 sqft',
       'Lot Size: 9278 sqft', '1,130 Square Feet', 'Lot Size: 0.66 acres',
       '1,302 Square Feet', '2,100 Square Feet', 'Lot Size: 8668 sqft',
       'Lot Size: 7431 sqft', '4,000 Square Feet', 'Lot Size: 3571 sqft',
       '4,788 Square Feet', '2,201 Square Feet', 'Lot Size: 10.48 acres',
       'Lot Size: 7100 sqft', 'Multi-Family', 'Lot Size: 11.40 acres',
       'Lot Size: 0.28 acres', '1,700 Square Feet', '3,504 Square Feet',
       'Lot Size: 1.62 acres'], dtype=

In [45]:
pricedf[pricedf['house_type'] == 'Lot Size: 9496 sqft']

,address,bathrooms,bedrooms,city_state_zip,house_type,price,sqft,url
135,403 Oertli Ln,"1,568 Square Feet",Multi-Family,"Austin, TX 78753",Lot Size: 9496 sqft,260000,2 Bathrooms,https://www.trulia.com/p/tx/austin/403-oertli-...


In [47]:
 def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [53]:
newpricedf = pricedf[pricedf['sqft'].map(lambda x: True if str(x).isdigit() else False)].copy()

In [59]:
newpricedf['price'] = newpricedf['price'].astype(int)
newpricedf['sqft'] = newpricedf['sqft'].astype(int)

In [61]:
newpricedf["price_per_sqft"] = newpricedf['price'] / newpricedf['sqft']

In [64]:
newpricedf.drop(['bathrooms', 'bedrooms', 'url', 'house_type'], axis=1, inplace=True)

In [73]:
newpricedf.drop_duplicates(inplace=True)

In [74]:
newpricedf.shape

(662, 5)

In [78]:
newpricedf.address.is_unique

False

In [79]:
address = newpricedf.address
newpricedf[address.isin(address[address.duplicated()])]

,address,city_state_zip,price,sqft,price_per_sqft
221,14921 Custis Ln,"Austin, TX 78725",241990,2040,118.622549
569,14921 Custis Ln,"Austin, TX 78725",241990,0,inf


In [80]:
newpricedf.drop(569, inplace=True)

In [81]:
newpricedf.address.is_unique

True

In [90]:
newpricedf.address = newpricedf.address.str.upper()